# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

filepath = '/content/drive/My Drive/netflix_titles.csv'

df = pd.read_csv(filepath)

In [12]:
#Erzeugung eines Datensatzes mit 20% der Daten
df = df.sample(int(df.shape[0]*0.2))
df.head(7)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,temp_genre
5073,60010836,Movie,Rocky III,Sylvester Stallone,"Sylvester Stallone, Talia Shire, Burt Young, C...",United States,"August 1, 2019",1982,PG,100 min,"Dramas, Sports Movies",After taking a pounding from a powerful young ...,Dramas
2410,80121845,TV Show,Love Is in the Air,NaN,"Murat Ünalmış, Birce Akalay, Selen Soyder, Erm...",NaN,"March 1, 2017",2010,TV-PG,1 Season,"International TV Shows, TV Dramas","Toprak, who lost her baby, is hired as a wet n...",International TV Shows
4712,80104240,Movie,Rainbow Time,Linas Phillips,"Linas Phillips, Melanie Lynskey, Timm Sharp, T...",United States,"December 1, 2016",2016,NR,92 min,"Comedies, Dramas, Independent Movies",A developmentally disabled man obsessed with t...,Comedies
657,80998890,Movie,Upstarts,Udai Singh Pawar,"Priyanshu Painyuli, Chandrachoor Rai, Shadab K...",India,"October 18, 2019",2019,TV-14,112 min,"Comedies, Dramas, International Movies","Hoping to do good while making millions, three...",Comedies
2204,80241616,Movie,Little Lunch: The Nightmare Before Graduation,Robyn Butler,"Flynn Curry, Olivia Deeble, Madison Lu, Oisín ...",Australia,"March 15, 2018",2016,TV-Y7,25 min,"Children & Family Movies, Comedies",Mrs. Gonsha's students are excited about the e...,Children & Family Movies
6029,70184127,TV Show,Big Bad Beetleborgs,NaN,"Wesley Barker, Herbie Baez, Elisabeth Lund, Sh...","United States, France, Japan","January 1, 2016",1997,TV-Y7-FV,2 Seasons,"Kids' TV, TV Comedies",When three teens free a spirit that offers to ...,Kids' TV
4461,80090672,Movie,Bob's Broken Sleigh,Jay Surridge,"Cole Howard, Bruce Greenwood, Victor Garber, M...",Canada,"December 17, 2015",2015,TV-G,47 min,Children & Family Movies,"An overly eager elf improves Santa's sleigh, b...",Children & Family Movies


In [84]:
#Splitten der Spalte genres in einzelne Genres
df['temp_genre'] = df['listed_in'].apply(lambda genres: genres.split(',')[0])
df['temp_genre']

5073                      Dramas
2410      International TV Shows
4712                    Comedies
657                     Comedies
2204    Children & Family Movies
                  ...           
2387                      Dramas
3582          Action & Adventure
4905                    Comedies
4378              Crime TV Shows
1719               Documentaries
Name: temp_genre, Length: 249, dtype: object

#Input of selected movie and number of recommendations

In [85]:
selected_movie = 'Rocky III'  #Input movie title
num_rec = 10 #Input number of recommendations

In [86]:
df1 = pd.DataFrame( np.concatenate( (df[['title', 'temp_genre']].values, df[['title', 'rating']].values), axis=0 ) )
df1.columns = ['title', 'value']
df1

,title,value
0,Rocky III,Dramas
1,Love Is in the Air,International TV Shows
2,Rainbow Time,Comedies
3,Upstarts,Comedies
4,Little Lunch: The Nightmare Before Graduation,Children & Family Movies
...,...,...
493,The Silence,TV-MA
494,The Mask of Zorro,PG-13
495,Otherhood,R
496,Demon's Path,TV-MA


In [87]:
df['title'].unique().shape[0] * df['temp_genre'].unique().shape[0] * df['rating'].unique().shape[0]

71136

In [88]:
# Graphen erstellen
#import networkx as nx
G = nx.from_pandas_edgelist(df1, source='title', target='value')

In [89]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 283
Number of edges: 498
Average degree:   3.5194


In [90]:
import matplotlib.pyplot as plt

#plt.figure(figsize=(20, 20))
#nx.draw_networkx(G, with_labels=False, alpha=0.5, node_size=50

In [91]:
[n for n in G.neighbors(selected_movie)]

['Dramas', 'PG']

In [92]:
i = 0
for neighbor in G.neighbors(selected_movie):
  print(neighbor)
  i += 1
  if i == 5:
    break

Dramas
PG


In [93]:
G.number_of_edges()

498

In [94]:
# Ermittlung Jaccard Coeffizient
jacc_gen = nx.jaccard_coefficient(G) #Generator für Jaccard Koeffizient

In [95]:
# Ermittlung Adamic Adar Coeffizient
adam_ad = nx.adamic_adar_index(G)  #Generator für Adamic_Adar_Index

In [96]:
jacc_gen

<generator object _apply_prediction.<locals>.<genexpr> at 0x7f01d6b29fc0>

In [97]:
jacc_dict = {}
for u,v,p in jacc_gen:
  jacc_dict[(u, v)] = p

In [98]:
adam_ad_dict = {}
for u,v,p in adam_ad:
  adam_ad_dict[(u, v)] = p

In [99]:
for key in jacc_dict:
  print(key, jacc_dict[key])
  break

('Michael Lost and Found', 'Personal Shopper') 0.0


In [100]:
for key in adam_ad_dict:
  print(key, adam_ad_dict[key])
  break

('Michael Lost and Found', 'Personal Shopper') 0


In [101]:
results_adam = {}
for key in adam_ad_dict:
  if selected_movie in key:
    results_adam[key] = adam_ad_dict[key]

In [102]:
import operator
results_adam_sorted = sorted(results_adam.items(), key=operator.itemgetter(1), reverse=True)

In [103]:
results_adam_sorted[:num_rec]

[(('A.X.L.', 'Rocky III'), 0.5581106265512472),
 (('The Flintstones', 'Rocky III'), 0.5581106265512472),
 (('G-Force', 'Rocky III'), 0.5581106265512472),
 (('The Lamb', 'Rocky III'), 0.5581106265512472),
 (('Rocky III', "Legend of the Guardians: The Owls of Ga'Hoole"),
  0.5581106265512472),
 (('Personal Shopper', 'Rocky III'), 0.28853900817779266),
 (('The Discovery', 'Rocky III'), 0.28853900817779266),
 (('Madras Café', 'Rocky III'), 0.28853900817779266),
 (('Luck by Chance', 'Rocky III'), 0.28853900817779266),
 (('Teen Patti', 'Rocky III'), 0.28853900817779266)]

In [104]:
results_jacc = {}
for key in jacc_dict:
  if selected_movie in key:
    results_jacc[key] = jacc_dict[key]

In [105]:
import operator
results_jacc_sorted = sorted(results_jacc.items(), key=operator.itemgetter(1), reverse=True)

results_jacc_sorted

[(('Personal Shopper', 'Rocky III'), 0.3333333333333333),
 (('The Discovery', 'Rocky III'), 0.3333333333333333),
 (('Madras Café', 'Rocky III'), 0.3333333333333333),
 (('Luck by Chance', 'Rocky III'), 0.3333333333333333),
 (('Teen Patti', 'Rocky III'), 0.3333333333333333),
 (('A.X.L.', 'Rocky III'), 0.3333333333333333),
 (('The Lion Woman', 'Rocky III'), 0.3333333333333333),
 (('ROMA', 'Rocky III'), 0.3333333333333333),
 (('The Flintstones', 'Rocky III'), 0.3333333333333333),
 (('The Chosen Ones', 'Rocky III'), 0.3333333333333333),
 (('Devrai', 'Rocky III'), 0.3333333333333333),
 (('The lynx', 'Rocky III'), 0.3333333333333333),
 (('Max Rose', 'Rocky III'), 0.3333333333333333),
 (('Being Mrs Elliot', 'Rocky III'), 0.3333333333333333),
 (('The Drug King', 'Rocky III'), 0.3333333333333333),
 (('Miss J Contemplates Her Choice', 'Rocky III'), 0.3333333333333333),
 (('Counterfeiting in Suburbia', 'Rocky III'), 0.3333333333333333),
 (('Jindua', 'Rocky III'), 0.3333333333333333),
 (('Dil Hai T

In [106]:
import numpy as np
erg_jacc = results_jacc_sorted[:num_rec]

erg_jacc_only = [tup[1] for tup in erg_jacc]
erg_jacc_only

erg_jacc_recom = [tup[0] for tup in erg_jacc ]
erg_jacc_rec =[tup[0] if tup[0]!=selected_movie else tup[1] for tup in erg_jacc_recom ]
erg_jacc_rec

erg_jacc_array = np.column_stack((erg_jacc_rec, erg_jacc_only))
erg_jacc_array

set_jacc = set(erg_jacc_rec)

set_jacc #Ergebnis als Menge

erg_jacc_top = [tup[1] for tup in erg_jacc]

erg_jacc_array

array([['Personal Shopper', '0.3333333333333333'],
       ['The Discovery', '0.3333333333333333'],
       ['Madras Café', '0.3333333333333333'],
       ['Luck by Chance', '0.3333333333333333'],
       ['Teen Patti', '0.3333333333333333'],
       ['A.X.L.', '0.3333333333333333'],
       ['The Lion Woman', '0.3333333333333333'],
       ['ROMA', '0.3333333333333333'],
       ['The Flintstones', '0.3333333333333333'],
       ['The Chosen Ones', '0.3333333333333333']], dtype='<U32')

In [107]:
erg_adam = results_adam_sorted[:num_rec]

erg_adam_only = [tup[1] for tup in erg_adam]

erg_adam_recom = [tup[0] for tup in erg_adam ]
erg_adam_rec =[tup[0] if tup[0]!=selected_movie else tup[1] for tup in erg_adam_recom ]

erg_adam_array = np.column_stack((erg_adam_rec, erg_adam_only))
erg_adam_array

set_adam = set(erg_adam_rec)

erg_adam_array



array([['A.X.L.', '0.5581106265512472'],
       ['The Flintstones', '0.5581106265512472'],
       ['G-Force', '0.5581106265512472'],
       ['The Lamb', '0.5581106265512472'],
       ["Legend of the Guardians: The Owls of Ga'Hoole",
        '0.5581106265512472'],
       ['Personal Shopper', '0.28853900817779266'],
       ['The Discovery', '0.28853900817779266'],
       ['Madras Café', '0.28853900817779266'],
       ['Luck by Chance', '0.28853900817779266'],
       ['Teen Patti', '0.28853900817779266']], dtype='<U45')

In [108]:
intersec = set_adam.intersection(set_jacc)
len(intersec)

7

#Final recommendations

In [109]:
intersec #Recommendations based on both measures (jacc and adam) 

{'A.X.L.',
 'Luck by Chance',
 'Madras Café',
 'Personal Shopper',
 'Teen Patti',
 'The Discovery',
 'The Flintstones'}

In [110]:
#Proportion of common top recommendations (jacc and adam) #with respect to chosen number of recommendations (num_rec)
prop_com_rec = len(intersec)/(num_rec)*100
print(f'Proportion of common top recommendations {prop_com_rec} %')

Proportion of common top recommendations 70.0 %
